# O Algoritmo Deutsch-Jozsa

## Uma Breve História:

## Explicação Teórica e Matemática:

Segundo o circuito, temos:

$$
|\psi_0\rangle = |0\rangle^{\otimes^n}|1\rangle
$$

Chamemos o vetor de 0s de $|x\rangle$

## Exemplo Simulado:

## Exemplo Real:

In [5]:
from qiskit import IBMQ
IBMQ.save_account('4854f480717fc1c972ca7c3020ed573ac7cf5d2eb91eccfdda886894c5da8b6c2fe7438ddcdc975df7bf0ce70ea008165054f48d9a34ad97286b94ac70196506')
IBMQ.load_accounts()
print("Backends disponíveis:")
IBMQ.backends()

Available backends:


[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx5') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQBackend('ibmq_qasm_simulator') from IBMQ()>]

In [6]:
from qiskit.backends.ibmq import least_busy

dispositivo = IBMQ.backends(filters = lambda x: x.configuration()['n_qubits'] > 3 and not x.configuration()['simulator'])

backend = least_busy(dispositivo)

print("O melhor é " + backend.name())

O melhor é ibmqx4


In [14]:
from qiskit import execute
shots = 1000
creditos = 3

tarefa = execute(qc, backend = backend, shots = shots, max_credits = creditos)

resultado = tarefa.result()

contador = resultado.get_counts(qc)
plot_histogram(contador)

NameError: name 'qc' is not defined

## Conclusões: